In [1]:
import sys
import os
from collections import defaultdict
import numpy as np
import pandas as pd
from scipy import stats
import re

In [2]:
cs_file = r"data\input\gradcafe_data-master\cs\cs_clean.csv"

df = pd.read_csv(cs_file, header=None)

all_columns = ["rowid", "uni_name", "major", "degree", "season", "decision",
               "decision_method", "decision_date", "decision_timestamp", 
               "ugrad_gpa", "gre_verbal", "gre_quant", "gre_writing", "is_new_gre", "gre_subject", 
               "status", "post_data", "post_timestamp", "comments"]

df.columns = all_columns

In [4]:
len(df["uni_name"])

27822

In [7]:
columns = ['uni_name', 'major', 'degree', 'season', 'decision', 'ugrad_gpa', 'gre_verbal', 'gre_quant', 
           'gre_writing', 'is_new_gre', 'gre_subject', 'status']

df.head()

,rowid,uni_name,major,degree,season,decision,decision_method,decision_date,decision_timestamp,ugrad_gpa,gre_verbal,gre_quant,gre_writing,is_new_gre,gre_subject,status,post_data,post_timestamp,comments
0,0,Tufts University,CS,PhD,S16,Accepted,E-mail,16-11-2015,1.447650e+09,3.6,166.0,163.0,4.5,True,NaN,American,16-11-2015,1447650000,Met with professor beforehand. 1 year academic...
1,1,SUNY Stony Brook,CS,MS,S16,Rejected,E-mail,14-11-2015,1.447477e+09,3.5,144.0,167.0,3.0,True,NaN,International,14-11-2015,1447477200,non CS background
2,2,Columbia University,CS,MS,S16,Other,Other,14-11-2015,1.447477e+09,NaN,NaN,NaN,NaN,NaN,NaN,International,14-11-2015,1447477200,So there was no actual Nov 15 deadline? Or did...
3,3,Columbia University,CS,PhD,S16,Other,Other,13-11-2015,1.447391e+09,NaN,NaN,NaN,NaN,NaN,NaN,International,13-11-2015,1447390800,I haven't found the website with the deadline ...
4,4,Columbia University,CS,MS,S16,Other,Other,13-11-2015,1.447391e+09,NaN,NaN,NaN,NaN,NaN,NaN,Other,13-11-2015,1447390800,Thanks guys! Good to know I'm not the only one...


In [8]:
df.describe()

,rowid,decision_timestamp,ugrad_gpa,gre_verbal,gre_quant,gre_writing,gre_subject,post_timestamp
count,27822.000000,2.780500e+04,7222.000000,8493.000000,8493.000000,8248.000000,262.000000,2.782200e+04
mean,13910.500000,1.325873e+09,3.698129,288.676439,343.097021,3.863227,800.305344,1.326125e+09
std,8031.663931,7.862555e+07,0.646575,213.479164,277.912011,0.725768,90.468673,7.842295e+07
min,0.000000,4.013424e+08,0.900000,133.000000,136.000000,2.000000,200.000000,1.139375e+09
25%,6955.250000,1.267679e+09,3.500000,154.000000,163.000000,3.500000,750.000000,1.267679e+09
50%,13910.500000,1.331096e+09,3.700000,161.000000,168.000000,4.000000,820.000000,1.331226e+09
75%,20865.750000,1.394510e+09,3.870000,480.000000,740.000000,4.500000,860.000000,1.394770e+09
max,27821.000000,1.576386e+09,9.990000,800.000000,800.000000,6.000000,990.000000,1.447650e+09


In [39]:
def get_valid_data(tmp):
    return tmp[(tmp["ugrad_gpa"]>=0)
            & (tmp["ugrad_gpa"]<=4.0)
            & (tmp["gre_verbal"]>=130)
            & (tmp["gre_verbal"]<=170)
            & (tmp["gre_quant"]>=130)
            & (tmp["gre_quant"]<=170)
            & (tmp["gre_writing"]>=0.0)
            & (tmp["gre_writing"]<=6.0)
           ]

def get_percentile_score(tmp, my_ugrad_gpa, my_gre_verbal, my_gre_quant, my_gre_writing):  
    ugrad_gpa_x = stats.percentileofscore(tmp.ugrad_gpa, my_ugrad_gpa)
    gre_verbal_x = stats.percentileofscore(tmp.gre_verbal, my_gre_verbal)
    gre_quant_x = stats.percentileofscore(tmp.gre_quant, my_gre_quant)
    gre_writing_x = stats.percentileofscore(tmp.gre_writing, my_gre_writing)
    
    return ugrad_gpa_x, gre_verbal_x, gre_quant_x, gre_writing_x

def get_match_scores(df, uni_names, my_ugrad_gpa, my_gre_verbal, my_gre_quant, my_gre_writing):
    uni_match = {}
    
    for uni in uni_names:
        tmp = df[(df["uni_name"]==uni)]
        
        tmp = get_valid_data(tmp)
        
        ugrad_gpa_x, gre_verbal_x, gre_quant_x, gre_writing_x = get_percentile_score(tmp,
                                                                                       my_ugrad_gpa, 
                                                                                       my_gre_verbal, 
                                                                                       my_gre_quant, 
                                                                                       my_gre_writing)

        if len(tmp)>0:
            uni_match[uni] = (ugrad_gpa_x + gre_verbal_x + gre_quant_x + gre_writing_x)/4

        uni_match = sorted(uni_match.items(), key=lambda kv: kv[1], reverse=True)
        uni_match = dict(uni_match)
    
    return uni_match

def get_best_match_univ(uni_match, count):
    i = 0
    for uni in uni_match:
        print("{} : {}".format(uni, uni_match[uni]))
        if count>0 and i==count-1:
            break
        i += 1

In [40]:
def get_university_match_score(tmp, uni, my_ugrad_gpa, my_gre_verbal, my_gre_quant, my_gre_writing, details=False):
    tmp = tmp[(tmp["uni_name"]==uni)]
    tmp = get_valid_data(tmp)    
    
    print("Undergrad GPA")
    print("-"*30)
    print("Percentile             : {}%".format(stats.percentileofscore(tmp.ugrad_gpa, my_ugrad_gpa)))
    if details:
        print("Min                : {} ".format(tmp.ugrad_gpa.min()))
        print("Max                : {} ".format(tmp.ugrad_gpa.max()))
        print("Mean               : {} ".format(tmp.ugrad_gpa.mean()))
        print("Standard Deviation : {} ".format(tmp.ugrad_gpa.std()))

    print("\nGRE Verbal")
    print("-"*30)
    print("Percentile             : {}%".format(stats.percentileofscore(tmp.gre_verbal, my_gre_verbal)))
    if details:
        print("Min                : {} ".format(tmp.gre_verbal.min()))
        print("Max                : {} ".format(tmp.gre_verbal.max()))
        print("Mean               : {} ".format(tmp.gre_verbal.mean()))
        print("Standard Deviation : {} ".format(tmp.gre_verbal.std()))

    print("\nGRE Quant")
    print("-"*30)
    print("Percentile             : {}%".format(stats.percentileofscore(tmp.gre_quant, my_gre_quant)))
    if details:
        print("Min                : {} ".format(tmp.gre_quant.min()))
        print("Max                : {} ".format(tmp.gre_quant.max()))
        print("Mean               : {} ".format(tmp.gre_quant.mean()))
        print("Standard Deviation : {} ".format(tmp.gre_quant.std()))

    print("\nGRE Writing")
    print("-"*30)
    print("Percentile             : {}%".format(stats.percentileofscore(tmp.gre_writing, my_gre_writing)))
    if details:
        print("Min                : {} ".format(tmp.gre_writing.min()))
        print("Max                : {} ".format(tmp.gre_writing.max()))
        print("Mean               : {} ".format(tmp.gre_writing.mean()))
        print("Standard Deviation : {} ".format(tmp.gre_writing.std()))
    
    
def get_university_match_stat(tmp, uni):
    tmp = tmp[(tmp["uni_name"]==uni)]
    tmp = get_valid_data(tmp)    
    
    print("Undergrad GPA")
    print("-"*30)
    print("Min: {}".format(tmp.ugrad_gpa.min()))
    print("Max: {}".format(tmp.ugrad_gpa.max()))
    print("25%: {}".format(tmp.ugrad_gpa.quantile(0.25)))
    print("50%: {}".format(tmp.ugrad_gpa.quantile(0.5)))
    print("75%: {}".format(tmp.ugrad_gpa.quantile(0.75)))
    print("Mean: {}".format(tmp.ugrad_gpa.mean()))
    print("Std: {}".format(tmp.ugrad_gpa.std()))

    print("\nGRE Verbal")
    print("-"*30)
    print("Min: {}".format(tmp.gre_verbal.min()))
    print("Max: {}".format(tmp.gre_verbal.max()))
    print("25%: {}".format(tmp.gre_verbal.quantile(0.25)))
    print("50%: {}".format(tmp.gre_verbal.quantile(0.5)))
    print("75%: {}".format(tmp.gre_verbal.quantile(0.75)))
    print("Mean: {}".format(tmp.gre_verbal.mean()))
    print("Std: {}".format(tmp.gre_verbal.std()))

    print("\nGRE Quant")
    print("-"*30)
    print("Min: {}".format(tmp.gre_quant.min()))
    print("Max: {}".format(tmp.gre_quant.max()))
    print("25%: {}".format(tmp.gre_quant.quantile(0.25)))
    print("50%: {}".format(tmp.gre_quant.quantile(0.50)))
    print("75%: {}".format(tmp.gre_quant.quantile(0.75)))
    print("Mean: {}".format(tmp.gre_quant.mean()))
    print("Std: {}".format(tmp.gre_quant.std()))

    print("\nGRE Writing")
    print("-"*30)
    print("Min: {}".format(tmp.gre_writing.min()))
    print("Max: {}".format(tmp.gre_writing.max()))
    print("25%: {}".format(tmp.gre_writing.quantile(0.25)))
    print("50%: {}".format(tmp.gre_writing.quantile(0.50)))
    print("75%: {}".format(tmp.gre_writing.quantile(0.75)))
    print("Mean: {}".format(tmp.gre_writing.mean()))
    print("Std: {}".format(tmp.gre_writing.std()))

In [41]:
status_list = ['American', 'International', 'International with US Degree', 'Other', np.nan]
decision_list = ['Accepted', 'Interview', 'Other', 'Rejected', 'Wait listed', np.nan]
uni_names = list(set(df.uni_name))
uni_names.sort()
uni_names

['ABC',
 'AT&T Labs Fellowship Program (ALFP)',
 'Aalto University',
 'Aberystwyth University',
 'Al.I.Cuza University',
 'Aligarh Muslim University',
 'All University Applicant',
 'American University Of Beirut',
 'Amherst College',
 'Anna University',
 'Arizona State University (ASU)',
 'Auburn University',
 'Austin, Cornell, Gatech, UMich, UToronto',
 'Austin, Cornell, Gatech, UMinn, TAMU, Stonybrook',
 'Bangladesh University Of Enginnering And Technology',
 'Bank Street College Of Education',
 'Baylor University',
 'Beijing Institute Of Technology',
 'Birla Institute Of Technology And Science (BITS)',
 'Boston University (BU)',
 'Brandeis University',
 'Brigham Young University',
 'Brock University',
 'Brown University',
 'CSUN',
 'CUNY',
 'Cal Poly Pomona',
 'Cal Poly San Luis Obispo',
 'California Institute Of Technology (Caltech)',
 'California School Of Professional Psychology',
 'California State University, Chico',
 'California State University, Fullerton',
 'California State

In [42]:
major = "CS"
degree = "PhD"
decision = "Accepted"
is_new_gre = True
status1 = 'International'
status2 = 'International with US Degree'
# season = "F15"


my_ugrad_gpa = 3.80
my_gre_verbal = 152
my_gre_quant = 162
my_gre_writing = 3.0

In [44]:
tmp = df[(df["major"]==major)
         & (df["degree"]==degree)
         & (df["decision"]==decision)
         & (df["is_new_gre"]==is_new_gre)
         & ((df["status"]==status1) | (df["status"]==status2))
#          & (df["season"]==season)
        ]

In [46]:
uni_match = get_match_scores(tmp, uni_names, my_ugrad_gpa, my_gre_verbal, my_gre_quant, my_gre_writing)

In [47]:
count = 100

get_best_match_univ(uni_match, count)

Florida Institute Of Technology : 100.0
North Dakota State University : 100.0
Towson University : 100.0
University Of Louisiana, Lafayette : 100.0
University Of Nebraska, Lincoln : 100.0
University Of North Carolina, Charlotte : 100.0
University Of Peradeniya, Sri Lanka : 100.0
Michigan Technological University (MTU) : 93.75
University Of Arkansas, Fayetteville : 93.75
University Of Kentucky : 87.5
Colorado School Of Mines (CSM) : 75.0
Florida State University (FSU) : 75.0
Louisville : 75.0
Michigan State University (MSU) : 75.0
Ohio State University, Columbus (OSU) : 75.0
Texas Tech University : 75.0
University Of California, Merced : 75.0
University Of Houston : 75.0
University Of Kansas : 75.0
University Of Louisville : 75.0
University Of New Mexico (UNM) : 75.0
University Of North Texas : 75.0
University Of Texas, Arlington : 75.0
University Of Texas, El Paso : 75.0
University Of Texas, San Antonio (UTSA) : 75.0
West Virginia University : 75.0
University Of Nevada, Reno : 72.5
Iowa

In [52]:
uni = "Stanford University"
# uni = "Cornell University"
# uni = "Massachusetts Institute Of Technology (MIT)"

In [50]:
get_university_match_score(tmp, uni, my_ugrad_gpa, my_gre_verbal, my_gre_quant, my_gre_writing, details=False)

Undergrad GPA
------------------------------
Percentile             : 38.46153846153846%

GRE Verbal
------------------------------
Percentile             : 15.384615384615385%

GRE Quant
------------------------------
Percentile             : 15.384615384615385%

GRE Writing
------------------------------
Percentile             : 7.6923076923076925%
